In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# 1. Fetch data
ticker = 'AAPL'
start_date = '2015-01-01'
end_date = '2024-12-31'
data = yf.download(ticker, start=start_date, end=end_date)
data = data[['Close']]

# 2. Add indicators
data['SMA_20'] = data['Close'].rolling(window=20).mean()
data['SMA_50'] = data['Close'].rolling(window=50).mean()

# RSI
window_length = 14
delta = data['Close'].diff()
up = delta.clip(lower=0)
down = -1 * delta.clip(upper=0)
average_gain = up.rolling(window=window_length).mean()
average_loss = down.rolling(window=window_length).mean()
rs = average_gain / average_loss
data['RSI'] = 100 - (100 / (1 + rs))

# 3. Normalize close price for LSTM
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[['Close']].dropna())

# 4. Prepare data for LSTM
window_size = 60
X, y = [], []
for i in range(window_size, len(scaled_data)):
    X.append(scaled_data[i - window_size:i])
    y.append(scaled_data[i])
X, y = np.array(X), np.array(y)

# 5. Build and train LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X.shape[1], 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X, y, epochs=10, batch_size=32)

# 6. Save model in Keras format
model.save('lstm_model.keras')


/var/folders/0h/2xx1qtq90mb55c65vg8njg4m0000gn/T/ipykernel_7065/4089751435.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed

Epoch 1/10



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0322
Epoch 2/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.3250e-04
Epoch 3/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 3.2025e-04
Epoch 4/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 3.1158e-04
Epoch 5/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 2.8556e-04
Epoch 6/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 3.0381e-04
Epoch 7/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.5880e-04
Epoch 8/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.5719e-04
Epoch 9/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.3849e-04
Epoch 10/10
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 2.5341e-04
